# RFSoC Offload Overlay
## Board Notebook

---
<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

## Overview
> The RFSoC offload overlay features high-speed data transfer between the RFSoC4x2 board and a computer via the QSFP28 port. QSFP28 allows speeds up to 100 Gbps which enables data to be transferred and processed off-device directly from the RF-ADCs at RF data rates. The offload overlay also contains a data switch and a PS to PL DMA allowing custom payloads to be sent from 
the boards OS to the client. Figure 1 shows a high level overview diagram of the RFSoC4x2 offload overlay.


<figure>
<img src='./assets/rfsoc_offload_arch.svg' width='80%'/>
<figcaption><b>Figure 1: The architecture overview of the RFSoC offload design.</b></figcaption>
</figure>

## Aims
* To introduce the RFSoC offload overlay.
* To send RF ADC data from the RFSoC to the PC/Client using QSFP.
* To send data generated in PS to the PC/Client using QSFP.

## Last Revised
* 08/07/22 - Updated code and text
* 30/06/22 - Initial revision
---

## Hardware Setup
For this demonstration, your RFSoC4x2 development board should be connected in loopback, with DAC B connected to ADC B. The DAC can be used to send test signals to the ADC using a signal generator.

* Connect DAC B to ADC A, as shown in Figure 2.
* Install the QSFP network interface card in your PC and setup the neccessery drivers*.
* Insert the optical transceiver modules to the RFSoC4x2 board and the PCs QSFP network card.
* Insert the fiber optic cable to both optical transceiver modules.

\* Refer to the project README for more extensive hardware & driver setup.

<figure>
<img src='./assets/hw_setup.svg' width='70%'/>
<figcaption><b>Figure 2: Hardware setup for RFSoC data offload to PC .</b></figcaption>
</figure>

Make sure the switches on RFSoC 4x2 are set to:
* PL_SW3 - Off
* PL_SW2 - Off
* PL_SW1 - Off
* PL_SW0 - On

As shown in Figure 3

<figure>
<img src='./assets/rfsoc_switches.png' height='80%' width='80%'/>
<figcaption><b>Figure 3: RFSoC4x2 hardware switch configuration. .</b></figcaption>
</figure>

<div class="alert alert-heading alert-danger">
<b>Warning:</b> In this demo signals are transmitted via SMA cables. This device can also transmit wireless signals. Unlicensed transmission of wireless signals may be illegal in your location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. If you are unsure, please seek professional support.
</div>

---

This overlay requires two notebooks to operate: a board notebook (this one), to be run on the RFSoC board, and a client notebook, which must be run on the PC/server that is connected to the board via the QSFP28 connection.

Before we start, make sure to download the `rfsoc_offload_client.ipynb` notebook supplied with this overlay, and open it on your client computer with Jupyter-Lab. Additionally, a QSFP setup script is supplied, `qsfp_setup.sh`, that will help configure the Mellanox NIC and, if required, should be downloaded onto the client computer as well.

We will be jumping between the board and client notebooks throughout this example, so it is best to have them both open in separate browser windows. In this notebook any commands that are required to be run on the client side will be preceded by an <span style="color:orange">**orange**</span> alert box, while any commands to be run on the board will be preceded by a <span style="color:green">**green**</span> alert box. All main section headings are identical between board and client notebooks and are numbered to make it easier to follow through the different steps.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the board.
</div> 

## 1. Board Setup

First we need to download the overlay to the PL

In [ ]:
from rfsoc_qsfp_offload.overlay import Overlay
ol = Overlay(ignore_version=True)

### Configure CMAC

The CMAC IP encapsulates the UltraScale+ Integrated 100G Ethernet Subsystem and provides Ethernet Media Access Controller (MAC), Physical Coding Sublayer (PCS) and Reed-Solomon Forward Error Correction (RS-FEC) functionality.

First of all we need to turn on the Forward Error Correction on the CMAC. To do this we set a couple of registers using MMIO.

In [ ]:
ol.cmac.mmio.write(0x107C, 0x3) # RSFEC_CONFIG_ENABLE
ol.cmac.mmio.write(0x1000, 0x7) # RSFEC_CONFIG_INDICATION_CORRECTION

Then we can start the Xilinx 100 GbE CMAC core.

In [ ]:
ol.cmac.start()

---

<div class="alert alert-box alert-warning">
The following steps are required to be executed on the client.
</div> 

## 2. Client Setup

Follow the instructions in the client notebook to set up the QSFP network settings.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the board.
</div> 

## 3. Configuring the Overlay

### Setup Netlayer IP

The Netlayer IP converts axi stream data into UDP packets and allows talking to other network devices.
To learn more about Netlayer IP refer to [XUP Vitis Network example README](https://github.com/Xilinx/xup_vitis_network_example).

First we setup the QSFP IP address for the board.

In [ ]:
board_ip = '192.168.4.99'
ol.netlayer.set_ip_address(board_ip, debug=True)

Next we set up a socket and populate the socket table with the relevant information.

In [ ]:
client_ip = '192.168.4.1'
ol.netlayer.sockets[0] = (client_ip, 60133, 60133, True)

ol.netlayer.populate_socket_table()

### Set Data Source

This design allows for data to be sent aross the QSFP network from either the RF-ADC, or sent directly from the PS. An AXI Stream switch facilitates this functionality.

Choose the data source to be sent over the QSFP network.

* 0 - Data from the PS to PL DMA
* 1 - Data acquired from RF-ADC

Set the data source as RF-ADC.

In [ ]:
ol.source_select(1) # 0 - DMA | 1 - RF-ADC

### Setup the RF Data Converters

Since we're sending data sent from the RF-DAC and received from the RF-ADC, we need to configure the RF data converters.

First we configure the RF-ADC channel.

In [ ]:
ADC_TILE = 2       # ADC Tile 226
ADC_BLOCK = 0       # ADC Block 0
ADC_SAMPLE_FREQUENCY = 4915.2  # MSps
ADC_PLL_FREQUENCY    = 491.52   # MHz
ADC_FC = -1228.8 # Centering around middle of sample rate

ol.initialise_adc(tile=ADC_TILE,
                  block=ADC_BLOCK,
                  pll_freq=ADC_PLL_FREQUENCY,
                  fs=ADC_SAMPLE_FREQUENCY,
                  fc=ADC_FC)

Then we configure the RF-DAC channel.

In [ ]:
DAC_TILE = 0       # DAC Tile 228
DAC_BLOCK = 0       # DAC Block 0
DAC_SAMPLE_FREQUENCY = 4915.2  # MSps
DAC_PLL_FREQUENCY = 491.52   # MHz
DAC_FC = 0.0

ol.initialise_dac(tile=DAC_TILE,
                  block=DAC_BLOCK,
                  pll_freq=DAC_PLL_FREQUENCY,
                  fs=DAC_SAMPLE_FREQUENCY,
                  fc=DAC_FC
                 )

### Enable Packet Generator
The Netlayer IP requires a `tlast` signal to signify the end of a packet. Since the RF-ADC AXI-Stream signal does not contain a `tlast` we need to generate one ourselves using a packet generator IP. 

The following cell sets the size of a data packet to be sent over the network.
To reach the maximum throughput of the system, jumbo Ethernet frames can be used but need to be enabled on all devices in the network. Maximum Transmission Unit (MTU) for Ethernet is typically 1500 bytes, but jumbo frames can reach payloads of up to 9000 bytes.

The CMAC Tx interface is 512 bits (64 bytes) wide. Thus to enable jumbo frames set the packet size to 128. 

If jumbo frames are **not** available on your client then use a packet size of 16 instead.

In [ ]:
ol.packet_generator.packetsize = 128 # 128 * 64 bytes = 8192 bytes to be sent

Now we can enable the packet generator.

In [ ]:
ol.packet_generator.enable()

---

<div class="alert alert-box alert-warning">
The following steps are required to be executed on the client.
</div> 

## 4. Receving RF Data

Run the appropriate cells on the client notebook to start receving data from the RF-ADC. As there is nothing yet being sent to the RF-ADC no signals will be present.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the board.
</div> 

## 5. Generating a Signal
For this part of the demo we drive the RF-DAC as stimulus for the RF-ADC. We first generate a signal in this notebook and pass it to PS memory via an AXI DMA. The DMA is set up in Cyclic BD mode which allows the DMA to loop through the data in memory, sending it to the RF-DAC, with no further interaction required from the user.

First we need to import the signal generator driver.

In [ ]:
from rfsoc_offload import signal_generator

The RF-DAC is set up with an output sample rate of 4915.2 Msps and 4x interpolation. This means that the signal we generate requires a sample rate of $f_{s DAC}/4$. 

Next we can generate a 81.6 MHz sine wave using the `signal_generator` module supplied with this design.

In [ ]:
sin_data = signal_generator.sine(f=81.6e6, fs=DAC_SAMPLE_FREQUENCY*1e6/4)
sin_data = signal_generator.convert_to_int16(sin_data)

We can then use plotly to check the signal is correct.

In [ ]:
import plotly.graph_objs as go
go.FigureWidget(go.Scatter(y=sin_data[0:256]))

Since the RF-DAC uses the complex mixer, it expects an interleaved I/Q signal at its input. Since our sine wave is real only (i.e. only an I signal), we can interleave zero values as the Q inputs.

In [ ]:
import numpy as np

zero_data = np.zeros(sin_data.size, np.int16)
interleaved = np.empty((sin_data.size + zero_data.size,), dtype=sin_data.dtype)
interleaved[0::2] = sin_data
interleaved[1::2] = zero_data

Finally, we can transfer the generated sine wave to memory using the DMA.

In [ ]:
from pynq import allocate

tx_buffer = allocate(shape=(interleaved.size,), dtype=np.int16)
tx_buffer[:] = interleaved
ol.axi_dma_dac.sendchannel.transfer(tx_buffer, cyclic=True)

You should now be able to see the generated signal on the client notebook.

<div class="alert alert-heading alert-danger">
    <b>Warning:</b> If you want to send additional data to the DMA it <b>must</b> be stopped before running the above cell again. Not doing so will hang the DMA and the board will require a power-cycle before it will be operational again. We show the code for the stop function in a few cells below.
</div>

### Frequency Sweep using the NCO

We can use the RF-DAC NCO to sweep the sine wave across the spectrum. By running the next cell you should see the signal move incremently up the spectrum on the client notebook. This process will take approximately 10 seconds.

You will notice that the signal changes from a single to a double tone in the spectrum. This is due to the carrier mixing with the sine wave in a process called Double Sideband Suppressed Carrier Amplitude Modulation (AM-DSB-SC).

In [ ]:
import xrfdc
from tqdm.notebook import trange # progress bar

f_start = 0 # MHz
f_stop = 2000 # MHz

for i in trange(f_start, f_stop, 10):
    ol.rfdc.dac_tiles[0].blocks[0].MixerSettings['Freq'] = i
    
# Set the NCO back to 0 Hz.
ol.rfdc.dac_tiles[0].blocks[0].MixerSettings['Freq'] = 0

### Shutdown the Packet Generator and DMA

Make sure to stop the packet generator and DMA by running the cell below

In [ ]:
ol.packet_generator.disable()
ol.axi_dma_dac.sendchannel.stop()

---

<div class="alert alert-box alert-warning">
The following steps are required to be executed on the client.
</div> 

## 6. Stop Receiving Data

Before going onto the next section, remember to stop grabbing data on the client first.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the board.
</div> 

## 7. Sending Data From the PS

We can also send data directly from the PS to the CMAC without the need for the RF-DAC.

First we switch to the PS as the data source using the AXI-Stream Switch.

In [ ]:
ol.source_select(0) # 0 - DMA | 1 - RFDC

### Sending a Ramp Signal

First we generate a simple ramp function using NumPy.

In [ ]:
data = np.arange(738, dtype=np.int16)

We can then plot the ramp signal for inspection.

In [ ]:
import plotly.graph_objs as go
go.FigureWidget(go.Scatter(y=data))

Finally we send it to the CMAC and network interface via a DMA transaction.

In [ ]:
out_buffer = allocate(shape=(len(data),), dtype=np.int16)
out_buffer[:] = data

ol.axi_dma_cmac.sendchannel.transfer(out_buffer)
ol.axi_dma_cmac.sendchannel.wait()

Head over to the RFSoC QSFP offload client notebook to inspect the results.

---

<div class="alert alert-box alert-warning">
The following steps are required to be executed on the client.
</div> 

## 8. Receving Data from the PS

Finally, switch over to the RFSoC QSFP offload client notebook to receive and plot the ramp function.

---

## Using GNU Radio

We also supply a method of receving and visualising the data using GNU Radio. To do this please refer to the GNU Radio setup guide in the [GitHub repository](https://github.com/strath-sdr/rfsoc_qsfp_offload/tree/master/gnuradio).

## Conclusion

This notebook has presented the RFSoC offload design in action. It has shown:
* The CMAC & networklayer IP configuration.
* Send real time data aquired by the RF-ADC trough QSFP network interface.
* Send data generated in PS over the QSFP network.